# namespace

スコープ定義  
* tf.name_scope() : 汎用的なスコープ定義 （TensorBoardへの出力は、この識別子設定を利用） 
* tf.variable_scope() : 変数を管理するためのスコープ定義  
 
変数定義  
* tf.Variable() : 低レベルの変数定義
* tf.get_variable() : 変数スコープを考慮した変数定義（tf.Variableの高レベルAPI）   
 
tf.get_variable() は，変数名の識別子（新規か？重複がないか？）を管理しながら，変数の定義を行います．必ず tf.variable_scope() とセットで使います．

In [1]:
import tensorflow as tf

### スコープ定義なし

まずは、スコープ定義を利用せず変数を定義してみます。 
変数は、  
* tf.get_variable  
* tf.Variable  

の２つの方法で定義しています。

変数のスコープ名を確認してみると、当然ながらスコープ名は定義されていません。

In [2]:
g1 = tf.get_variable("g1", [1], dtype=tf.float32)
g2 = tf.Variable(1, name="g2", dtype=tf.float32)
v = tf.add(g1, g2)

# スコープ名と変数名を表示
print(g1.name)  # g1:0
print(g2.name)  # g2:0
print(v.name)   # Add:0

g1:0
g2:0
Add:0


同じ変数を定義するとどのようになるかを試してみます。  
* tf.Variable　-> 新しい変数（○○_1）が定義される
* tf.get_variable ->　同スコープで同じ変数を定義するとエラーとなる

tf.get_variableの場合、「同じ識別子で変数を取るのはおかしい」というエラーが発生します。

In [3]:
#g1 = tf.get_variable("g1", [1], dtype=tf.float32) -> error
g2 = tf.Variable(1, name="g2", dtype=tf.float32)
v = tf.add(g1, g2)
print(g1.name)  # g1:0
print(g2.name)  # g2_1:0
print(v.name)   # Add_1:0

g1:0
g2_1:0
Add_1:0


### tf.name_scope() によるスコープ定義

次に、tf.name_scope()　でスコープを定義してみます。

結果をみると、
* tf.Variable　-> スコープ定義が適用されている
* tf.get_variable -> スコープ定義が適用されていない

という違いが確認されました。

In [4]:
# tf.name_scope
with tf.name_scope("name_sp"):
    x1 = tf.get_variable("x1", [1], dtype=tf.float32)
    x2 = tf.Variable(1, name="x2", dtype=tf.float32)
    y = tf.add(x1, x2)

print(x1.name)  # x1:0
print(x2.name)  # name_sp/x2:0
print(y.name)   # name_sp/Add:0

x1:0
name_sp/x2:0
name_sp/Add:0


同じスコープ定義内で同じ変数を定義すると次のような結果になります。

* tf.Variable　-> 新しいスコープ定義（○○_1）で変数が定義される
* tf.get_variable ->　同スコープで同じ変数を定義するとエラーとなる

tf.Variableは、スコープ定義がない場合と異なり、新しいスコープが定義されるようです。

In [5]:
# tf.name_scope
with tf.name_scope("name_sp"):
    #x1 = tf.get_variable("x1", [1], dtype=tf.float32)
    x2 = tf.Variable(1, name="x2", dtype=tf.float32)
    y = tf.add(x1, x2)

print(x1.name)  # x1:0
print(x2.name)  # name_sp_1/x2:0
print(y.name)   # name_sp_1/Add:0

x1:0
name_sp_1/x2:0
name_sp_1/Add:0


### tf.variable_scope によるスコープ定義

tf.variable_scope の場合、tf.get_variable で定義した変数にもスコープ定義が適用されます。


In [6]:
with tf.variable_scope("name_sp"):
    x1 = tf.get_variable("x1", [1], dtype=tf.float32)  
    x2 = tf.Variable(1, name="x2", dtype=tf.float32)
    y = tf.add(x1, x2)

print(x1.name)  # name_sp/x1:0
print(x2.name)  # name_sp_2/x2:0
print(y.name)   # name_sp_2/Add:0

name_sp/x1:0
name_sp_2/x2:0
name_sp_2/Add:0


tf.get_variableで以前定義した変数を再利用する場合、　　
tf.variable_scope の引数に"reuse=True"を設定します。

tf.Variable では、新しいスコープが定義されてしまっています。

In [7]:
with tf.variable_scope("name_sp", reuse=True):
    x1 = tf.get_variable("x1", [1], dtype=tf.float32)  
    x2 = tf.Variable(1, name="x2", dtype=tf.float32)
    y = tf.add(x1, x2)

print(x1.name)  # name_sp/x1:0
print(x2.name)  # name_sp_3/x2:0
print(y.name)   # name_sp_3/Add:0

name_sp/x1:0
name_sp_3/x2:0
name_sp_3/Add:0
